In [29]:
import os

current_dir = os.getcwd()

mitsuba_file = os.path.join(
    current_dir,
    "central_park",
    "central_park.xml",
)

In [30]:
import drjit as dr
import mitsuba as mi
mi.set_variant("cuda_ad_rgb")

scene = mi.load_file(mitsuba_file)

params = mi.traverse(scene)
print(params)

SceneParameters[
  ------------------------------------------------------------------------------------------------
  Name                                         Flags    Type  Parent
  ------------------------------------------------------------------------------------------------
  Camera.near_clip                                      float PerspectiveCamera
  Camera.far_clip                                       float PerspectiveCamera
  Camera.shutter_open                                   float PerspectiveCamera
  Camera.shutter_open_time                              float PerspectiveCamera
  Camera.film.size                                      ScalarVector2u HDRFilm
  Camera.film.crop_size                                 ScalarVector2u HDRFilm
  Camera.film.crop_offset                               ScalarPoint2u HDRFilm
  Camera.x_fov                                 ∂, D     Float PerspectiveCamera
  Camera.to_world                              ∂, D     Transform4f PerspectiveC

In [31]:
# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import os # Configure which GPU
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e) # Avoid warnings from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

tf.random.set_seed(1) # Set global random seed for reproducibility

In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, Paths2CIR

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [33]:
# Load integrated scene
scene = load_scene(mitsuba_file)

In [34]:
################################# Configure Rx mobility parameters #############

rx_3D_object_name = "mesh-Cube"
rx_current_x = -181
rx_current_y = 146
rx_current_z = 15

################################# Configure Tx parameters #############
# Ground
tx_x = -66
tx_y = 106
tx_z = 63

nTx = 64
nRx = 4

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(
    num_rows=int(np.sqrt(nTx)),
    num_cols=int(np.sqrt(nTx)),
    vertical_spacing=0.5,
    horizontal_spacing=0.5,
    pattern="tr38901",
    polarization="V",
)

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(
    num_rows=int(np.sqrt(nRx)),
    num_cols=int(np.sqrt(nRx)),
    vertical_spacing=0.5,
    horizontal_spacing=0.5,
    pattern="tr38901",
    polarization="V",
)

# Create transmitter
tx = Transmitter(
    name="tx",
    position=[tx_x, tx_y, tx_z],
    orientation=[-2.0944,0.261799,0]
)

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver

rx = Receiver(
    name="rx",
    position=[rx_current_x, rx_current_y, rx_current_z - 10],
    orientation=[-0.523599,0,0]
)

# Add receiver instance to scene
scene.add(rx)

scene.frequency = 40e9  # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = True  # If set to False, ray tracing will be done per antenna element (slower for large arrays)
# cm = scene.coverage_map(max_depth=5,
#                     cm_cell_size=(3., 3.), # Grid size of coverage map cells in m
#                     combining_vec=None,
#                     precoding_vec=None,
#                     num_samples=int(1e2))
# starting_instant = time.time()
# Compute propagation paths
paths = scene.compute_paths(
    max_depth=5,
    method="stochastic",  # For small scenes the method can be also set to "exhaustive"
    num_samples=1e6,  # Number of rays shot into random directions, too few rays can lead to missing paths
    seed=1,
)  # By fixing the seed, reproducible results can be ensured
# ending_instant = time.time()
# print(f"RT duration: {ending_instant-starting_instant}")

# cam_angle_x = 250
# cam_angle_y = -220
# cam_angle_z = 0
# # Create new camera with different configuration
# my_cam = Camera("my_cam", position=[
#     rx_starting_x, 
#     rx_starting_y, 
#     rx_starting_z],
#     look_at=[cam_angle_x,
#             cam_angle_y,
#             cam_angle_z])
# scene.add(my_cam)

# scene.render(camera="my_cam", num_samples=512);


jit_flush_malloc_cache(): Dr.Jit exhausted the available memory and had to flush its allocation cache to free up additional memory. This is an expensive operation and will have a negative effect on performance. You may want to change your computation so that it uses less memory. This warning will only be displayed once.


In [ ]:
scene.preview(paths=paths, show_devices=True, show_paths=True, show_orientations=True)

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …